# Spiking multicompartment PC network

## Abstract
Predictive coding is a promising theoretical framework for understanding the hierarchical sensory processing in the brain, yet how it is implemented with cortical spiking neurons is still unclear. While most existing works have taken a hand-wiring approach to creating microcircuits which match experimental results, recent work in applying the optimisation approach revealed that cortical connectivity might result from self-organisation given some fundamental computational principle, ie. energy efficiency. We thus investigated whether predictive coding properties in a multicompartment spiking neural network can result from energy optimisation. We found that only the model trained with an energy objective in addition to a task-relevant objective was able to reconstruct internal representations given top-down expectation signals alone. Neurons in the energy-optimised model also showed differential responses to expected vs unexpected stimuli, qualitatively similar to experimental evidence for predictive coding. These findings indicated that predictive-coding-like behaviour might be an emergent property of energy optimisation, providing a new perspective on how predictive coding could be achieved in the cortex.

In [8]:
%load_ext autoreload
%autoreload 2

import torch
from predcoding.snn.network import EnergySNN
from predcoding.snn.network_old import SnnNetwork3Layer
from predcoding.training import get_stats_named_params
from predcoding.utils import count_parameters

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# set seed
torch.manual_seed(999)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu


In [9]:
# network parameters
use_alif_neurons = True  # whether use adaptive neuron or not
clf_alpha = 1
energy_alpha = 0.05  # - config.clf_alpha
spike_alpha = 0.0  # energy loss on spikes
num_readout = 10
one_to_one = True
lr = 1e-3
alg = "fptt"
p_dropout = 0.4
is_recurrent = False
b_0 = 0.1      # neural threshold baseline
R_m = 3         # membrane resistance
gamma = 0.5     # gradient scale
lens = 0.5
baseline_threshold = b_0

# training parameters
T = 50
K = 10  # k_updates is num updates per sequence
omega = int(T / K)  # update frequency
clip = 1.0
log_interval = 20
epochs = 35
alpha = 0.2
beta = 0.5
rho = 0.0 

In [17]:
# set input and t param
d_in = 784
d_hidden = [600, 500, 500]
n_classes = 10

# define network
model = EnergySNN(
    d_in,
    d_hidden,
    d_out=n_classes,
    is_adaptive=use_alif_neurons,
    one_to_one=one_to_one,
    p_dropout=p_dropout,
    is_recurrent=is_recurrent,
    b0=b_0,
    device=device,
)

model_old = SnnNetwork3Layer(
    d_in,
    d_hidden,
    n_classes,
    is_adapt=use_alif_neurons,
    one_to_one=one_to_one,
    dp_rate=p_dropout,
    is_rec=is_recurrent,
)

# define new loss and optimiser
print(f"total param count {count_parameters(model)}")
print(f"total param count {count_parameters(model_old)}")

print(get_stats_named_params(model)["input_layer.weight"][3:])
# print(get_stats_named_params(model_old).keys())

bias set to 0
total param count 2450020
total param count 2455520
(tensor([[0., -0., 0.,  ..., 0., 0., -0.],
        [-0., 0., 0.,  ..., -0., -0., 0.],
        [-0., 0., -0.,  ..., -0., 0., 0.],
        ...,
        [-0., 0., 0.,  ..., 0., 0., -0.],
        [-0., 0., -0.,  ..., -0., -0., -0.],
        [-0., 0., -0.,  ..., -0., -0., 0.]]),)
